### Steps for finding the home/work location 

In [ ]:
#Importing libraries
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon

In [20]:
import os
os.getcwd()
os.chdir(r'C:\Users\Kasutaja\Desktop\PhD_task_helsinki_University\test')

In [21]:
# Reading from a sample data
df = pd.read_csv('cdr2.csv')

In [22]:
# Splitting time column into two columns - month and hour
df['time'] = pd.to_datetime(df['time'])
df['month'] = df['time'].dt.month
df['hour'] = df['time'].dt.hour

In [24]:
## Visualizing sample data 
df.head()

,time,pos_usr_id,X,Y,month,hour
0,2008-04-01 08:40:45,John,25.422217,59.455275,4,8
1,2008-04-01 08:56:34,John,25.355000,59.476381,4,8
2,2008-04-01 09:08:27,John,25.422217,59.455275,4,9


### Finding Home/work location
<blockquote> Finding the  home / work location based on different temporal variations :
             Group data (Twitter User IDs) based on the desired temporal variation in this case is daily.</blockquote>

### Tracking cross-border country movement 
<blockquote> Finding the most frequent tweets based on the holidays (summer/winter), to assign them as 
tourist and aggreate the data with landmarks of the specific country.
we can select (June - August ), (december-January)  for summer and winter holidays respectively. 
for instance : Summer holiday = selected months from the df_summer = df[df['month'] == 6].</blockquote>



In [13]:
def site_location(data):
    data['site'] = str(round(data['X'], 3)) +' '+ str(round(data['Y'], 3))
    return data
    
def day_night_time(dataset, period):
    if period == 'daytime':
        data[period] = data['hour'].apply(lambda x: 1 if (x >= 9 and x <= 18) else 0)
    elif period == 'nighttime':
        data[period] = data['hour'].apply(lambda x: 1 if (x >= 20 and x <= 7) else 0)
    df1 = data[data[period] == 1] 
    df2 = df1.groupby(['pos_usr_id', 'site'])[period].sum().reset_index().sort_values(period, ascending = False)
    users = df2['pos_usr_id'].unique()
    user_site = {}
    for i in users:
        subset_data = df2[df2['pos_usr_id'] == i].head(1)
        user_site[i] = subset_data['site'].values
    return pd.DataFrame.from_dict(user_site, orient='index').reset_index().rename(columns = {'index':'user', 0:'site'})

# By changing the ('NAME_1') to another ('NAME_0') the destination location based on the county, city level will be reveal.

def country_name(data, country_data):
    data['X_Y_coord'] = data[['X', 'Y']].apply(Point, axis = 1)
    data['country'] = ''
    n_country = country_data.shape[0]
    n_data = data.shape[0]
    for i in range(n_country):
        for j in range(n_data):
            if country_data['geometry'][i].contains(data['X_Y_coord'][j]):
                data['country'][j] = country_data['NAME_1'][i]
    return data

In [15]:
data = df.apply(site_location, axis = 1)
data.head()

,time,pos_usr_id,X,Y,month,hour,site
0,2008-04-01 08:40:45,John,25.422217,59.455275,4,8,25.422 59.455
1,2008-04-01 08:56:34,John,25.355000,59.476381,4,8,25.355 59.476
2,2008-04-01 09:08:27,John,25.422217,59.455275,4,9,25.422 59.455


In [16]:
# DAY_TIME : Work location 
day_night_time(data, 'daytime')

,user,site
0,John,25.422 59.455


<blockquote> In this section the Home/work location in a municipality can be achieved by adding an extra column to the dataset </blockquote>

<blockquote> Task 2 : for finding the destination country and count the number of tweets grouped by country inorder to find out how long they have spend time there we can use the same function as TASK 1 but instead of work location we can track the movement from home to the destination countries.</blockquote>

In [17]:
country_data = gpd.read_file('gadm36_EST_1.shp')

In [18]:
country_name(data, country_data)

<ipython-input-13-1b631efcab70>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['country'][j] = country_data['NAME_1'][i]


,time,pos_usr_id,X,Y,month,hour,site,daytime,X_Y_coord,country
0,2008-04-01 08:40:45,John,25.422217,59.455275,4,8,25.422 59.455,0,POINT (25.4222166 59.455275),Harju
1,2008-04-01 08:56:34,John,25.355000,59.476381,4,8,25.355 59.476,0,POINT (25.355 59.4763805),Harju
2,2008-04-01 09:08:27,John,25.422217,59.455275,4,9,25.422 59.455,1,POINT (25.4222166 59.455275),Harju
